# High Dimensional Weighted Gene Correlation Network Analysis (hdWGCNA)

**Authorship:**
Adam Klie, *08/24/2022*
***
**Description:**
Notebook to run a WGCNA on stimulated pancreatic islet multiome data (well just the RNA)
***

In [2]:
# Conversion libraries and Seurat
library(SeuratDisk)
library(SeuratData)
library(Seurat)
library(Signac)

# plotting and data science packages
library(tidyverse)
library(cowplot)
library(patchwork)

# co-expression network analysis packages:
library(WGCNA)
library(hdWGCNA)

# using the cowplot theme for ggplot
theme_set(theme_cowplot())

# set random seed for reproducibility
set.seed(12345)

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

Attaching SeuratObject

Attaching sp

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ purrr::reduce() masks Signac::reduce()


Attaching package: ‘patchwork’


The following object is masked from ‘package:cowplot’:

    align_plots


Loading required package: dynamicTreeCut

Loading required package: fastcluster


Attaching package: ‘fastcluster’


The following object is masked from ‘package:stats’:

    hclust





Attaching package: ‘WGCNA’


The following object is masked from ‘package:stats’:

    cor




In [3]:
# Parameters
CELL.TYPES <- "beta"  # Meaning α, β, δ in this context
ASSAY <- "RNA"
NORMALIZATION <- "ND"
NN <- 25
GENES <- "all"

In [4]:
# File names for saving
NAME=paste0(CELL.TYPES, "cells_", GENES, "genes_", NORMALIZATION, "_", NN, "neighbors")
OUT= file.path("./basic_visualization", NAME)
OUT

[1] "./basic_visualization/betacells_allgenes_ND_25neighbors"

In [5]:
# load the preprocessed RDS from 1_network_construction.ipynb
seurat_obj <- readRDS(sprintf('network_construction/%s_hdWGCNA.rds', NAME))

In [10]:
seurat_obj@misc$betacells_allgenes_ND_25neighbors$

[1] "betacells_allgenes_ND_25neighbors"

# Basic Visualization

In [6]:
# make a featureplot of hMEs for each module
options(repr.plot.width=12, repr.plot.height=12)
png(sprintf("%s_ModuleFeaturePlot.png", OUT), widt=1200, height=1200)
plot_list <- ModuleFeaturePlot(
  seurat_obj,
  reduction="umap.wnn",
  features='MEs', # plot the hMEs
  order=TRUE # order so the points with highest hMEs are on top
)

# stitch together with patchwork
wrap_plots(plot_list, ncol=3)
dev.off()

[1] "betacells_allgenes_ND_25neighbors-M1"
[1] "betacells_allgenes_ND_25neighbors-M2"
[1] "betacells_allgenes_ND_25neighbors-M3"
[1] "betacells_allgenes_ND_25neighbors-M4"
[1] "betacells_allgenes_ND_25neighbors-M5"


png 
  2

In [7]:
# compute gene scoring for the top 25 hub genes by kME for each module
# with Seurat method
seurat_obj <- ModuleExprScore(
  seurat_obj,
  n_genes = 25,
  method='Seurat'
)

Selecting by kME_betacells_allgenes_ND_25neighbors-M1
Selecting by kME_betacells_allgenes_ND_25neighbors-M2
Selecting by kME_betacells_allgenes_ND_25neighbors-M3
Selecting by kME_betacells_allgenes_ND_25neighbors-M4
Selecting by kME_betacells_allgenes_ND_25neighbors-M5


In [ ]:
head(seurat_obj@misc$allcells_ND$module_scores)

In [ ]:
head(seurat_obj@misc$allcells_ND$MEs)

In [ ]:
# make a featureplot of hub scores for each module
options(repr.plot.width=12, repr.plot.height=12)
png(sprintf("%s_ModuleScorePlot.png", OUT), widt=1200, height=1200)
plot_list <- ModuleFeaturePlot(
  seurat_obj,
  reduction="umap.wnn",
  features='scores', # plot the hub gene scores
  order=TRUE, # order so cells are shuffled
  ucell = FALSE # depending on Seurat vs UCell for gene scoring
)

# stitch together with patchwork
wrap_plots(plot_list, ncol=3)
dev.off()

In [ ]:
# plot module correlagram
options(repr.plot.width=12, repr.plot.height=12)
png(sprintf("%s_ModuleCorrelation.png", OUT), widt=1200, height=1200)
ModuleCorrelogram(seurat_obj)
dev.off()

---

# Scratch

In [12]:
gene_info <- seurat_obj@misc$betacells_allgenes_ND_25neighbors$wgcna_modules

In [20]:
gene_info

,gene_name,module,color,kME_grey,kME_betacells_allgenes_ND_25neighbors-M1,kME_betacells_allgenes_ND_25neighbors-M2,kME_betacells_allgenes_ND_25neighbors-M3,kME_betacells_allgenes_ND_25neighbors-M4,kME_betacells_allgenes_ND_25neighbors-M5
,<chr>,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AL627309.5,AL627309.5,grey,grey,0.01621312,0.006754910,0.0279640506,0.010020962,0.03037640,-0.00120792
LINC01409,LINC01409,grey,grey,-0.05604869,0.008689548,-0.0343847019,0.072409864,0.06123836,0.08382473
LINC01128,LINC01128,betacells_allgenes_ND_25neighbors-M1,turquoise,0.12696136,-0.038470429,0.0388936024,-0.011345708,0.04271950,-0.07873651
SAMD11,SAMD11,betacells_allgenes_ND_25neighbors-M1,turquoise,0.13351788,-0.031510842,0.0499140776,-0.095160340,-0.12148682,-0.13311908
NOC2L,NOC2L,betacells_allgenes_ND_25neighbors-M1,turquoise,0.07246652,-0.016091112,0.0109280094,-0.025332913,-0.02473053,-0.05995336
AGRN,AGRN,betacells_allgenes_ND_25neighbors-M1,turquoise,0.11104950,-0.056400150,-0.0111848507,-0.031303693,-0.05214087,-0.09816540
C1orf159,C1orf159,betacells_allgenes_ND_25neighbors-M1,turquoise,0.17288954,-0.093383597,-0.0345580246,-0.027013551,-0.06209958,-0.14426726
AL390719.2,AL390719.2,betacells_allgenes_ND_25neighbors-M1,turquoise,0.08850812,-0.072449985,-0.0327315478,-0.019562254,-0.06723702,-0.08686925
TTLL10,TTLL10,betacells_allgenes_ND_25neighbors-M1,turquoise,0.11125938,-0.077815923,-0.0455817083,0.014582972,-0.03125120,-0.09232658


In [15]:
green_genes <- gene_info[gene_info$color == "green", ]

In [18]:
ord <- order(green_genes$`kME_betacells_allgenes_ND_25neighbors-M4`)

In [19]:
green_genes[ord, ]

,gene_name,module,color,kME_grey,kME_betacells_allgenes_ND_25neighbors-M1,kME_betacells_allgenes_ND_25neighbors-M2,kME_betacells_allgenes_ND_25neighbors-M3,kME_betacells_allgenes_ND_25neighbors-M4,kME_betacells_allgenes_ND_25neighbors-M5
,<chr>,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
HDAC9,HDAC9,betacells_allgenes_ND_25neighbors-M4,green,-0.20062408,0.5669551,0.4465884,-0.33573816,-0.0789643568,0.13572249
PDE3A,PDE3A,betacells_allgenes_ND_25neighbors-M4,green,-0.23603772,0.6478374,0.4885545,-0.37662928,-0.0777244584,0.18280617
ARHGAP24,ARHGAP24,betacells_allgenes_ND_25neighbors-M4,green,-0.12189260,0.4841849,0.3712696,-0.37741967,-0.0718240427,0.08507664
KCNS3,KCNS3,betacells_allgenes_ND_25neighbors-M4,green,-0.08780701,0.3695943,0.2729596,-0.23761936,-0.0707713435,0.04559347
MSC-AS1,MSC-AS1,betacells_allgenes_ND_25neighbors-M4,green,-0.07194113,0.2578816,0.2208840,-0.16625708,-0.0644743418,0.03594997
NKAIN2,NKAIN2,betacells_allgenes_ND_25neighbors-M4,green,-0.13275453,0.3908086,0.3056210,-0.18279323,-0.0621658583,0.11854762
LRRC4C,LRRC4C,betacells_allgenes_ND_25neighbors-M4,green,-0.17724749,0.4326693,0.2785103,-0.22549377,-0.0567708402,0.12701039
LINC02143,LINC02143,betacells_allgenes_ND_25neighbors-M4,green,-0.21032676,0.2966200,0.1406450,-0.12337849,-0.0566912991,0.15899277
MIR137HG,MIR137HG,betacells_allgenes_ND_25neighbors-M4,green,-0.20186034,0.5631838,0.4887578,-0.36539160,-0.0438346782,0.14406509
